## Libraries and imports

In [1]:
#analysis 
from datetime import date


import numpy as np
import pandas as pd
#graphing
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
from pathlib import Path
path_cwd=Path.cwd()

import sys
sys.path.append(str(path_cwd)+'/Functions')
import functions_reading,functions_plotting,functions_data,functions_calculations

from functions_reading import reading_hobo,reading_BGI,reading_JP,reading_RAD, reading_solcast
from functions_plotting import visually_compare, plot_ts
from functions_data import cut_merge,correlation_fun, replace_nans_list, replace_nans_one
from functions_calculations import vpd

from functools import reduce

## Reading + Processing

In [3]:
hobow_=reading_hobo() #Wasootch
BGI=reading_BGI() #BioGeoScience Institute
JP=reading_JP() #Jumpingpound

############# HOBO
#merging 2019-2022
dfs = [hobow_,BGI,JP]
HOBO_COMPLETE_all= reduce(lambda df_left,df_right: pd.merge(df_left, df_right,left_index=True, right_index=True,how='left'),dfs) 

#replace missing values from Wasootch with BGI and JP
HC=HOBO_COMPLETE_all.copy()
HOBO_COMPLETE=replace_nans_list(HC,cols_nan=['Rain(mm)','Temp(C)','RH','WS','GS','DP'],cols_replace=['Rain_BGI(mm)','Temp_BGI(C)','RH_JP','WS_JP','GS_JP','DP_JP'])

############# RADIATION 
#CAMPBELL RAD 
rad=(reading_RAD()).drop(columns=['NetRadAvg']) 

#BGI RAD
BGI_RAD=(HC.copy()).drop(columns=['Temp(C)','Rain(mm)','RH','WS','GS','DP','RH_JP','WS_JP','GS_JP','DP_JP','Temp_BGI(C)','Rain_BGI(mm)'])
rad_2=rad.merge(BGI_RAD,how='left', left_on=rad.index, right_on=BGI_RAD.index)
rad_2.index = pd.to_datetime(rad_2['key_0'])
RAD_COMPLETE_both=rad_2.drop(columns=['key_0'])

#Replace NaNs
RAD_COMPLETE=replace_nans_one(RAD_COMPLETE_both,'NetRad','Rad_BGI')

############# WEATHER 
WEATHER=(cut_merge(all_dfs=[HOBO_COMPLETE,RAD_COMPLETE])).drop(columns=['Rad_BGI'])

############# SOLCAST
GHI = reading_solcast()
#crop GHI and WEATHER to same time period
ghi=GHI.loc[WEATHER.index[0]:WEATHER.index[-1]]
weather=WEATHER.loc[ghi.index[0]:ghi.index[-1]]

## Calculations

### Emissivity
Atmospheric emissivity through potential and measured Solar radiation

In [4]:
#ASTRAL
from astral import LocationInfo
from astral.sun import sun
from astral.sun import zenith

l = LocationInfo()
l.name = 'Kananaskis'
l.region = 'Alberta'
l.latitude = 50.970535344046056
l.longitude = -115.0950050354004
l.timezone = 'Canada/Mountain'#Etc/GMT-1'
l.elevation = 1443

#GENERAL
albedo=0.2 #absorptivity=(1-albedo)
sigma=5.67*10**(-8) #Stefan-Boltzmann constant W/m2K4
R=8.314 #Gas constant J/molK
M_w=0.018 #molecular weight of water kg/mol


In [5]:
#FUNCTIONS

# Vapor concentration of air g/m3 --> Cva=mass of water vapour per unit volume of air; mass wv=no of moles of wv * molecular weight of wv
# M_w*n/V=M_w*P/(R*T); P-saturated vapour pressure, R-gas constant, T-temperature, M_w-molecular weight of water
def Cva(T,RH):
    cv_sat = 6.11*np.exp((17.27*T)/(T+237.3))*1.e3*(M_w/(R*(T+273.15)))
    cv = cv_sat*RH
    return cv

# Atmospheric emissivity calculated with solar declination
def atm_emissivity(measuredradiation,day,T,RH):
    sin_sd = 0.3985 * np.sin(4.869 + 0.0172 * day + 0.03345 * np.sin(6.224 + 0.0172 * day)) #sd- solar declination
    cos_sd = np.sqrt(1 - sin_sd ** 2)
    h_s = np.arccos(-np.tan(l.latitude) * (sin_sd/cos_sd))
    potentialRadiation = 117.5*(1.e6*(h_s*np.sin(l.latitude)*sin_sd+np.cos(l.latitude)*cos_sd*np.sin(h_s))/np.pi) #radiant flux density W/m2, 117.5 is the amount of energy on a 1m2 surface perpendiculat to the sun
    T_t=measuredradiation/potentialRadiation
    c_l=2.33-3.33*T_t
    c_l = np.where(c_l < 0., 0., c_l)
    c_l = np.where(c_l > 1., 1., c_l)
    c_va=Cva(T,RH)*1.e-3 #g/m3
    epsilon_a=0.58*np.power(c_va,1/7)
    emissivity = (1-0.84*c_l)*epsilon_a+0.84*c_l # eq 15.4^ and 15.5 in Bittelli et al 2020 
    return emissivity

# Atmospheric emissivity calculated with zenith angle
def emissivity_alt(measuredradiation,df,T,RH): #same as above but with directly from zenith angle from astral
    dff = df.copy()
    idx = dff.index
    Z = []
    for i in range(len(dff)):
        z = zenith (l.observer, idx[i])
        Z.append(z)

    dff['zenith']=Z
    dff['Qp'] = 1360*np.cos(np.degrees(dff['zenith']))
    potentialradiation = dff['Qp'].groupby(pd.Grouper(freq='24H')).transform('sum') 
    T_t=measuredradiation/potentialradiation
    c_l=2.33-3.33*T_t
    c_l = np.where(c_l < 0., 0., c_l)
    c_l = np.where(c_l > 1., 1., c_l)
    c_va=Cva(T,RH)*1.e-3 #g/m3
    epsilon_a=0.58*np.power(c_va,1/7)
    emissivity = (1-0.84*c_l)*epsilon_a+0.84*c_l
    return emissivity


In [6]:
emissivity = atm_emissivity(ghi['GHI'],ghi.index.dayofyear,weather['Temp(C)'],weather['RH'])
